# PALM, pylm y mastr

## Guillem Borrell Nogueras

## 17 de agosto de 2016

# Pylm #

Pylm es la implementación en Python de PALM, un framework (librería) para crear clusters de microservidores de altas prestaciones. Consta de dos niveles de abstracción

1. Una API de alto nivel con servidores tipo que uno puede extender.

2. Una API de bajo nivel para desarrollar servidores al uso.

Esta presentación cubre el uso de la API de alto nivel.

El tipo más sencillo de servidor en pylm se comunica sólo con un cliente

![](./fig/standalone_single.png)

Y también es sencillo crear uno

In [ ]:
from pylm.standalone import Server

class MyServer(Server):
    def foo(self, message):
        self.logger.warning('Got a message')
        return b'you sent me ' + message
    
server = MyServer('my_server', 'tcp://127.0.0.1:5555')
server.start()

In [ ]:
from pylm.standalone import Client

client = Client('tcp://127.0.0.1:5555', 'my_server')
result = client.job('foo', b'a message')
print('Client got: ', result)

$> python client.py

Client got: b'you sent me a message'

# Lo que acaba de pasar #

1. Hemos utilizado un servidor modelo de tipo *Server* para crear nuestro *MyServer*

2. Hemos definido una función *foo* en *MyServer*

3. Hemos arrancado el nuevo servidor en el puerto 5555.

4. El cliente ha llamado la función *foo* y ha recogido el resultado.

# Las capacidades de Pylm #

* Del mismo modo que tenemos la clase *Server* disponemos de otras clases de servidores que soportan otras topologías de comunicación

* Cada servidor puede ser serial o paralelo

  * Serial: el servidor es un único proceso
  
  * Paralelo: el servidor tiene una arquitectura master-worker.
  
* Múltiples modelos de conexión

  * Standalone: conexión con un cliente
  
  * Chained: conexión con otros servidores
  
  * Pipelined: conexiones libres y manejados por la infraestructura

# Un servidor standalone paralelo #

![](./fig/standalone_parallel.png)

In [1]:
from pylm.standalone import Master

server = Master(name='server',
                pull_address='tcp://127.0.0.1:5555',
                push_address='tcp://127.0.0.1:5556',
                worker_pull_address='tcp://127.0.0.1:5557',
                worker_push_address='tcp://127.0.0.1:5558',
                db_address='tcp://127.0.0.1:5559',
                palm=True)


In [8]:
from pylm.standalone import Worker
import sys

class MyWorker(Worker):
    def foo(self, message):
        return self.name.encode('utf-8') + b' processed ' + message

server = MyWorker(sys.argv[1],
                  push_address='tcp://127.0.0.1:5558',
                  pull_address='tcp://127.0.0.1:5557',
                  db_address='tcp://127.0.0.1:5560')

In [4]:
from pylm.standalone import ParallelClient
from itertools import repeat

client = ParallelClient(push_address='tcp://127.0.0.1:5556',
                        pull_address='tcp://127.0.0.1:5555',
                        db_address='tcp://127.0.0.1:5559',
                        server_name='server')

if __name__ == '__main__':
    for response in client.job('foo', repeat(b'a message', 10), messages=10):
        print(response)

# Varios servidores seriales encadenados #

![](./fig/pipeline_single.png)

In [ ]:
from pylm.chained import Server

class Step1(Server):
    def foo(self, message):
        self.logger.warning('Got a message')
        return b'you sent me ' + message

server = Step1(name='step1', pull_address='tcp://127.0.0.1:5555',
               next_address='tcp://127.0.0.1:5557',
               next_call='step2.bar',
               db_address='tcp://127.0.0.1:5556')

In [ ]:
from pylm.chained import LastServer

class Last(LastServer):
    def baz(self, message):
        self.logger.warning('Got a message')
        return b'ACK: ' + message

server = Last(name='last', pull_address='tcp://127.0.0.1:5559',
              push_address='tcp://127.0.0.1:5561',
              db_address='tcp://127.0.0.1:5560')

In [ ]:
from pylm.chained import LoopClient

client = LoopClient(push_address='tcp://127.0.0.1:5561',
                    pull_address='tcp://127.0.0.1:5555',
                    db_address='tcp://127.0.0.1:5556',
                    server_name='step1')

if __name__ == '__main__':
    for result in client.job('foo', [b'a message'], messages=1):
        print('Client got: ', result)

# Servidores seriales y paralelos pueden combinarse #


![](./fig/chained_combined.png)


# Hay unos cuantos detalles relevantes #

* Toda la comunicación entre clientes y servidores, y entre masters y workers se hace por mensajes binarios.

* Pylm es un framework, proporciona las piezas para crear clusters de microservicios, pero no impone ninguna infraestructura (como Storm) ni ninguna topología de comunicación (como Spark)

* Pylm proporciona componentes para

  * Centralizar logs
  
  * Recoger información sobre el rendimiento del sistema (performance counters)
  
  * Proporcionar tolerancia a fallos para los workers
  
  * Persistencia en el master, en memoria y en disco.
  
  * ...

# Mastr #

Mastr utiliza Pylm para implementar el flujo valoración + agregación en paralelo

![](./fig/mastr.png)


# El flujo de datos en mastr #

1. El cliente de valoración recibe la información de la cartera

2. El master de valoración recibe la carga de trabajo, la distribuye entre los workers, y procesa el resultado

3. Una vez cada resultado es procesado, se manda al master de agregación.

4. La información de agregación se distribuye entre los workers.

5. Las operaciones de agregación se realizan mediante un cliente en tiempo real con los datos disponibles.

# Más detalles importantes #

* Los workers de valoración no tienen estado (si se caen no pasa nada). Los workers de agregación sí.

* La información de valoración debe transformarse a un archivo JSON que cumple ciertas condciones.

* Los resultados se van agregando a medida que están disponibles. No es necesario esperar a tener todas las valoraciones para realizar una agregación.

* Los workers de agregación pueden realizar operaciones que requiere comunicación entre ellos (shuffle)

# Conclusiones #

* La API de alto nivel de pylm es funcional. Aunque la API está en $\beta$.

* Sirve para crear servidores, masters, workers y clientes con topologías de comunicación sencillas.

* Es fácil de usar y ha demostrado tener un rendimiento excelente.

* Mastr sugiere que pylm tiene mucho potencial.

**Scais está siendo portado a pylm usando la API de bajo nivel**